In [1]:
# imports 
import cv2
import numpy as np 
import pandas as pd 
import ipywidgets as widgets
import mediapipe as mp
import tensorflow as tf
from mediapipe import solutions
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
from fastai.vision.all import show_image
from ipywidgets import interact, interactive, fixed, interact_manual
import import_ipynb
from landmarks import *
from decode_phrase import *

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
# mp_holistic = mp.solutions.holistic

importing Jupyter notebook from landmarks.ipynb
importing Jupyter notebook from decode_phrase.ipynb


In [2]:
# mediapipe hand detector (make sure hand_landmarker.task is in folder)
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, min_hand_detection_confidence=.5, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

#BaseOptions = mp.tasks.BaseOptions
#HandLandmarker = mp.tasks.vision.HandLandmarker
#HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
#VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the video mode:
#options = HandLandmarkerOptions( base_options=BaseOptions(model_asset_path='hand_landmarker.task'), running_mode=VisionRunningMode.VIDEO)
#detector = HandLandmarker.create_from_options(options)

def convert_to_mediapipe(path):
    
    ### initialize ###
    # open video 
    vid = cv2.VideoCapture(path)
    assert vid.isOpened(), 'Make sure that the video is in a format accepted by c2v.VideoCapture()' 
    
    # load first frame 
    boo, image = vid.read()
    assert boo, 'Unable to load first frame of video'
    
    ### loop over frames and apply mediapipe ### 
    count = 1
    images, landmarks = [], []
    while boo:
        
        # convert image to mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # change color from Blue,Green,Red to Red,Green,Blue
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        
        # use mediapipe to finc image
        result = detector.detect(mp_image)
        
        # cv2.imwrite("image"+str(count)+".jpeg", image)
        # add to lists 
        images += [image ]
        landmarks += [ result ]
        
        # update 
        boo, image = vid.read()
        count += 1
        
    return images, landmarks 
    

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
# TODO: current only has right hand
def process_video(self):
   
    L = []
    for frame in self.landmarks:
        if frame.hand_landmarks:
            
            # create the landmark
            x = landmark_pb2.LandmarkList()
            for v in frame.hand_landmarks[0]:
                x.landmark.add( x=v.x, y=v.y, z=v.z ) 
            
            # center the landmark
            x = center(x)
            
            # extract the data 
            data = np.array([ [ Landmark_vector(x,i) for i in range(1,21) ] ])
            
            # add to L
            L += [ data ]
        else:
            L += [ np.array([ [ np.array([0.,0.,0.]) for i in range(20) ] ]) ]   
    return L

In [4]:
#######################
##### Video class #####
#######################
                
### Video file Class
# set label = false if you do not want to automatically label the video
class video_file:           
    def __init__(self, path, label = True):
        
        # initialize
        frames, landmarkers = convert_to_mediapipe(path)
        
        # attributes
        self.path = path
        self.frames = frames 
        self.landmarks = landmarkers
        self.total_frames = len(frames)
        self.total_hand_frames = len([ x for x in self.landmarks if x.hand_landmarks])
        self.landmark_percentage = 0 if self.total_frames == 0 else self.total_hand_frames / self.total_frames
        
        # label video
        if label:
            self.hand_frames = process_video(self)
            phrase, labels = Corg(self.hand_frames)
            self.phrase = phrase
            self.labels = labels
            

    def __repr__(self): 
            a = " path: {0}\n Number of frames: {1}\n Percentage of frames with landmarks: {2}".format( self.path,
                                                                                                        self.total_frames, 
                                                                                                        self.landmark_percentage ) 

            return a 


In [5]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks])
        solutions.drawing_utils.draw_landmarks(
                                                annotated_image,
                                                hand_landmarks_proto,
                                                solutions.hands.HAND_CONNECTIONS,
                                                solutions.drawing_styles.get_default_hand_landmarks_style(),
                                                solutions.drawing_styles.get_default_hand_connections_style())

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image
  

In [6]:
def show_annotated_video(self):
    
    data = self.frames
    land = self.landmarks
    labels = self.labels
    phrase = self.phrase
    
    # show function
    def show_frame(i):
        
        # annotated image and show
        annotated_image = draw_landmarks_on_image(data[i], land[i])
        show_image(annotated_image, figsize=(6,6), title=f'Frame: {i} of {len(data)}   Phrase: {phrase}    Prediction: {labels[i]}')
    
    return show_frame

In [44]:
path = 'Samples/IMG_0958.MOV'
V = video_file(path)
print(V)

[(0, 107, 47), (107, 166, 40), (166, 233, 32), (233, 316, 49), (316, 399, 38), (399, 464, 40), (464, 519, 50)]
 path: Samples/IMG_0958.MOV
 Number of frames: 520
 Percentage of frames with landmarks: 1.0


In [45]:
f = show_annotated_video(V)
interact(f, i=widgets.IntSlider(min=0, max=len(V.frames)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))

interactive(children=(IntSlider(value=0, description='i', layout=Layout(width='1000px'), max=519), Output()), …

<function __main__.show_annotated_video.<locals>.show_frame(i)>

In [9]:
# model_path = '/Users/benjaminbreen/Desktop/test_hand.jpg'
# model_path = 

# Load the input image from an image file.
# mp_image = mp.Image.create_from_file(model_path)

# create img
# img = cv2.imread(model_path)

# STEP 2: Create an HandLandmarker object.
#base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
#options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
#detector = vision.HandLandmarker.create_from_options(options)

# STEP 3: Load the input image.
# image = mp.Image.create_from_file(model_path)
#mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)

# STEP 4: Detect hand landmarks from the input image.
#detection_result = detector.detect(mp_image)

In [47]:
# imports 
import math
import numpy as np
import tensorflow as tf
from spellchecker import SpellChecker
from Levenshtein import distance
from collections import Counter
from collections import defaultdict

# Labeling

# Encode
E = {" ":0, "!":1, "#":2, "$":3, "%":4, "&":5, "'":6, "(":7, ")":8, "*":9, "+":10, ",":11, "-":12, ".":13, 
          "/":14, "0":15, "1":16, "2":17, "3":18, "4":19, "5":20, "6":21, "7":22, "8":23, "9":24, ":":25, ";":26,
          "=":27, "?":28, "@":29, "[":30, "_":31, "a":32, "b":33, "c":34, "d":35, "e":36, "f":37, "g":38, "h":39,
          "i":40, "j":41, "k":42, "l":43, "m":44, "n":45, "o":46, "p":47, "q":48, "r":49, "s":50, "t":51, "u":52, 
          "v":53, "w":54, "x":55, "y":56, "z":57, "~":58}
# Decode
D = {j:i for i,j in E.items()}

# Encode - converts letter to number
def Encode(x):
    return E[x]

# Decode - converts number to letter
def Decode(x):
    return D[x]

def Corg(frames):
    
    # initialize 
    m = len(frames)
    thresh = .1 #threshold
    n = 59
    
    # load model 
    M = tf.keras.models.load_model('alphabetmodel.h5')
    X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ]) # Need softmax later for predictions
    
    # Predictions
    Predictions = []
    for x in frames:
        if np.any(x):
            Predictions.append( X.predict(np.array(x), verbose=0)[0] )
        else:
            Predictions.append( np.array( [0.] * 59 )  )
    
    ## score function 
    # Given a letter (e.g. 'a') and cluster label c (e.g. 0) returns a score of the letter for that clump 
    def score( i, letter ):
        s = Predictions[i][letter]
        return s
    

    ### Prefix sums
    D = {}
    for c in range(59):
        
        # initialize
        D[c] = defaultdict(int)
        pre, L = 0, []
        
        # loop
        for j in range(m):
            
            # update prefix sum
            pre += score(j,c)
            
            # loop over past prefix sum and compute score of i,j
            for i,x in enumerate(L):
                D[c][(i,j)] = (pre - x) * math.sqrt(j-i)
            
            # update list of prefixes
            L += [pre]
    
    ### Dynamic programming
    # dp array
    dp = defaultdict( lambda : (0,[]) )
    for i in range(m):
        
        if not np.any(Predictions[i]):
            continue
        
        # update past
        if dp[i][0] < dp[i-1][0]:
            dp[i] = dp[i-1]
        
        # update future
        for j in range(i+2, m):
            
            # loop
            for c in range(59):
                c_score = D[c][(i+1,j)]
                if dp[j][0] < c_score + dp[i][0]:
                     dp[j] = ( c_score + dp[i][0], dp[i][1] + [(i,j,c)] )
    
    # optimal score
    score, labs = dp[m-1]
    
    # new labels
    newlabels = ['?'] * m
    for i, j, letter in labs:
        for k in range(i,j+1):
            newlabels[k] = Decode(letter)
    
    # self.data_labels = newlabels
    print(labs)
            
    # sentence
    s = ''.join( Decode(i[2]) for i in labs)
    
    # return s, labs, D, [ Decode[np.argmax(i)] for i in Predictions]
    return s, newlabels

NameError: name 'spell' is not defined